Preprocessing

In [33]:
import pandas as pd
import numpy as np
import string
import re
import nltk

In [34]:
# Load dataset
def load_data():
    data = pd.read_csv('dataset1.csv')#ubah nama file sesai dengan nama file kalian
    return data

tweet_df = load_data()
tweet_df.head()

,datetime,username,content
0,2022-10-12 06:48:56+00:00,rxxyz6,@FiersaBesari terlalu menyakitkan ketika diliat
1,2022-10-12 06:28:05+00:00,nadubasyri,SEBUAH GARIS WAKTU...\n\nJika saatnya tiba sed...
2,2022-10-12 06:27:33+00:00,missheart222,@t @teddspotting @R @jericho_rosales @rajolaur...
3,2022-10-12 06:10:41+00:00,daissysfriend,"fiersa besari was right,\n\nkita adalah rasa y..."
4,2022-10-12 06:02:46+00:00,Selvi28874608,@FiersaBesari 🤣


In [35]:
#definisi dataframe
df  = pd.DataFrame(tweet_df[['datetime','username', 'content']])

In [36]:
#remove user 
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt    
df['remove_user'] = np.vectorize(remove_pattern)(df['content'], "@[\w]*")
df['remove_user']

0                      terlalu menyakitkan ketika diliat
1      SEBUAH GARIS WAKTU...\n\nJika saatnya tiba sed...
2       eddspotting          am_concepcion   eddspott...
3      fiersa besari was right,\n\nkita adalah rasa y...
4                                                      🤣
                             ...                        
995                                 Rada rada maksa yaa😭
996      semogaa remedial kali ini dapett hasil yang ...
997     Satru 2 - Denny caknan\nDan - So7\nRuntuh - F...
998                                 mungkin hrs remedial
999                     Ikut remedial juga boleh gk kak,
Name: remove_user, Length: 1000, dtype: object

In [37]:
#remove user
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt    
df['remove_user'] = np.vectorize(remove_pattern)(df['content'], "@[\w]*")

def remove(tweet):
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
    
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    return tweet
df['remove_http'] = df['remove_user'].apply(lambda x: remove(x))
df.sort_values("remove_http", inplace = True)
df.drop_duplicates(subset ="remove_http", keep = 'first', inplace = True)
df['remove_http'].to_csv('after_cleaning.csv',index=False)


In [38]:
#import stopword
from nltk.corpus import stopwords 
stopwords_indonesia = stopwords.words('indonesian')
 
#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#tokenize
from nltk.tokenize import TweetTokenizer

In [39]:
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)

In [40]:
def clean_tweets(tweet):
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
 
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    #remove coma
    tweet = re.sub(r',','',tweet)
    
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
     # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
 
    tweets_clean = []    
    for word in tweet_tokens:
        if (word not in stopwords_indonesia and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            stem_word = stemmer.stem(word) # stemming word
            tweets_clean.append(stem_word)
 
    return tweets_clean
df['tweet_clean'] = df['remove_http'].apply(lambda x: clean_tweets(x))

In [ ]:
#remove punct
def remove_punct(text):
    text  = " ".join([char for char in text if char not in string.punctuation])
    return text
df['Tweet'] = df['tweet_clean'].apply(lambda x: remove_punct(x))
 #=========================================================================#
df.sort_values("Tweet", inplace = True)
df.drop(df.columns[[0,1,2,3,4]], axis = 1, inplace = True)
df.drop_duplicates(subset ="Tweet", keep = 'first', inplace = True)
df.to_csv('output.csv',encoding='utf8', index=False)
df

,tweet_clean,Tweet
274,[],
55,"[abang, nggak, kenal, bimantara, humpuss, citr...",abang nggak kenal bimantara humpuss citra lamt...
649,"[abunawas, laen, jokowi boong, g, merugika, ra...",abunawas laen jokowi boong g merugika rakyat j...
578,"[abunawas, pas]",abunawas pas
626,"[adil, pertanggungjawaban, buka]",adil pertanggungjawaban buka
...,...,...
727,"[yth, presiden, kurang, inflasi, qbat, bbm, bb...",yth presiden kurang inflasi qbat bbm bbm tdk t...
348,"[yth, presiden, mohon, maaf, sih, urgensi, ban...",yth presiden mohon maaf sih urgensi bangun ikn...
751,"[yth, selamat, pilih, bpk, abdu, selamat, tuga...",yth selamat pilih bpk abdu selamat tugas
875,"[yuk, dkasih, ruang, utk, yg]",yuk dkasih ruang utk yg
